In [30]:
import pandas as pd

# Đường dẫn đến tệp Excel
file_path = './input/label.xlsx'

# Đọc tệp Excel
datatimeOA = pd.read_excel(file_path)

# Hiển thị các dòng đầu tiên của DataFrame
print(datatimeOA.head())
datatimeOA['StartDate'] = pd.to_datetime(datatimeOA['StartDate'], format='%H:%M:%S')

# Tính toán endDate bằng cách thêm duration (tính bằng giờ) vào StartDate
datatimeOA['endDate'] = datatimeOA['StartDate'] + pd.to_timedelta(datatimeOA['duration'], unit='s')

# Hiển thị các dòng đầu tiên của DataFrame sau khi tính toán
print(datatimeOA.head())


  StartDate  duration type
0  12:43:06     18.49   hy
1  12:44:55     13.36   hy
2  12:46:27     10.55   hy
3  12:46:41     10.60   hy
4  12:46:56     10.80   hy
            StartDate  duration type                 endDate
0 1900-01-01 12:43:06     18.49   hy 1900-01-01 12:43:24.490
1 1900-01-01 12:44:55     13.36   hy 1900-01-01 12:45:08.360
2 1900-01-01 12:46:27     10.55   hy 1900-01-01 12:46:37.550
3 1900-01-01 12:46:41     10.60   hy 1900-01-01 12:46:51.600
4 1900-01-01 12:46:56     10.80   hy 1900-01-01 12:47:06.800


In [31]:
datatimeOA['endDate'] = pd.to_datetime(datatimeOA['endDate'], format='%H:%M:%S:%f')

In [32]:
import pandas as pd

# Đường dẫn đến tệp Excel
file_path = './input/flowcmh20.xlsx'

# Đọc tệp Excel
flowData = pd.read_excel(file_path)



# Hiển thị các dòng đầu tiên của DataFrame
print(flowData.head())

           time     value
0  00:37:05:162  0.148586
1  00:37:05:167  0.148548
2  00:37:05:172  0.148485
3  00:37:05:177  0.148397
4  00:37:05:182  0.148284


In [33]:
def convert_hour_to_12(dt):
    if dt.hour == 0:
        return dt.replace(hour=12)
    return dt

In [34]:
flowData['time'] = pd.to_datetime(flowData['time'], format='%H:%M:%S:%f')
flowData['time'] = flowData['time'].apply(convert_hour_to_12)
flowData['nhan'] = 0
print(flowData.head())

                     time     value  nhan
0 1900-01-01 12:37:05.162  0.148586     0
1 1900-01-01 12:37:05.167  0.148548     0
2 1900-01-01 12:37:05.172  0.148485     0
3 1900-01-01 12:37:05.177  0.148397     0
4 1900-01-01 12:37:05.182  0.148284     0


In [35]:
for index, row in datatimeOA.iterrows():
    mask = (flowData['time'] >= row['StartDate']) & (flowData['time'] <= row['endDate'])
    flowData.loc[mask, 'nhan'] = 1
    

print(flowData.head())

                     time     value  nhan
0 1900-01-01 12:37:05.162  0.148586     0
1 1900-01-01 12:37:05.167  0.148548     0
2 1900-01-01 12:37:05.172  0.148485     0
3 1900-01-01 12:37:05.177  0.148397     0
4 1900-01-01 12:37:05.182  0.148284     0


In [36]:
flowData_nhan_1 = flowData[flowData['nhan'] == 1]
print(flowData_nhan_1)

                          time     value  nhan
72168  1900-01-01 12:43:06.002 -0.065932     1
72169  1900-01-01 12:43:06.007 -0.065745     1
72170  1900-01-01 12:43:06.012 -0.065540     1
72171  1900-01-01 12:43:06.017 -0.065319     1
72172  1900-01-01 12:43:06.022 -0.065085     1
...                        ...       ...   ...
628543 1900-01-01 01:29:27.877  0.044959     1
628544 1900-01-01 01:29:27.882  0.048235     1
628545 1900-01-01 01:29:27.887  0.051751     1
628546 1900-01-01 01:29:27.892  0.055507     1
628547 1900-01-01 01:29:27.897  0.059505     1

[361214 rows x 3 columns]


In [38]:
new_file_path = './output/flowData_modified.xlsx'

# Lưu DataFrame vào tệp Excel mới
flowData.to_excel(new_file_path, index=False)

In [64]:
import math
def create_training_data(data, window_size, step_size):

    _train = []
    _label = []

    for i in range(0, data.shape[0] - window_size, step_size):
        try:
            xs = data['value'].values[i: i + window_size]
        
            # Skip examples where the activity label changes within the window.
            if (data['nhan'][i+1] != data['nhan'][i + window_size-1]):
                print(f"Skipping index {i} due to label change within window.")
                continue

            label = data['nhan'][i + window_size-1]

            # Skip examples where the label is NaN.
            if math.isnan(label):
                print(f"Skipping index {i} due to NaN label.")
                continue

            _train.append(xs)
            _label.append(label)
        
        except Exception as e:
            print(f"An error occurred at index {i}: {e}")

    return _train, _label

In [59]:
data = pd.DataFrame({
    'value': [1, 2,3, 4, 5, 6],
    'nhan': [0, 0, 0, 1, 1, 1]
})

# Gọi hàm với window_size và step_size
window_size = 2
step_size = 1
_train, _test = create_training_data(data, window_size, step_size)

print("Training data:", _train)
print("Training labels:", _test)

Training data: [array([1, 2], dtype=int64), array([2, 3], dtype=int64), array([3, 4], dtype=int64), array([4, 5], dtype=int64)]
Training labels: [0, 0, 1, 1]


In [71]:

_train, _label = create_training_data(
    data=flowData, window_size=20, step_size=10)


Skipping index 72150 due to label change within window.
Skipping index 72160 due to label change within window.
Skipping index 75850 due to label change within window.
Skipping index 75860 due to label change within window.
Skipping index 93950 due to label change within window.
Skipping index 93960 due to label change within window.
Skipping index 96630 due to label change within window.
Skipping index 112350 due to label change within window.
Skipping index 112360 due to label change within window.
Skipping index 114460 due to label change within window.
Skipping index 114470 due to label change within window.
Skipping index 115150 due to label change within window.
Skipping index 115160 due to label change within window.
Skipping index 117270 due to label change within window.
Skipping index 117280 due to label change within window.
Skipping index 118150 due to label change within window.
Skipping index 118160 due to label change within window.
Skipping index 120310 due to label cha

In [72]:
print(_train[0])
print(len(_train))
print(len(_label))



[0.14858611 0.14854803 0.14848476 0.14839655 0.14828376 0.14814686
 0.14798641 0.14780309 0.14759765 0.14737093 0.14712388 0.14685753
 0.14657298 0.14627142 0.14595407 0.14562224 0.14527728 0.14492059
 0.14455364 0.14417789]
104433
104433


In [74]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(_train, _label, test_size=0.2, random_state=42)

In [100]:
import numpy as np


x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [103]:
print(x_train.shape)
print(y_train)

(83546, 20, 1)
[0 1 0 ... 0 0 1]


In [114]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# 3. Xây dựng mô hình CNN
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(20, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 4. Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 5. Huấn luyện mô hình
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
2089/2089 [==============================] - 5s 2ms/step - loss: 0.5595 - accuracy: 0.6760 - val_loss: 0.5573 - val_accuracy: 0.6785
Epoch 2/10
2089/2089 [==============================] - 4s 2ms/step - loss: 0.5466 - accuracy: 0.6930 - val_loss: 0.5459 - val_accuracy: 0.6978
Epoch 3/10
2089/2089 [==============================] - 5s 2ms/step - loss: 0.5404 - accuracy: 0.6995 - val_loss: 0.5456 - val_accuracy: 0.6962
Epoch 4/10
2089/2089 [==============================] - 5s 2ms/step - loss: 0.5389 - accuracy: 0.6992 - val_loss: 0.5426 - val_accuracy: 0.7047
Epoch 5/10
2089/2089 [==============================] - 5s 2ms/step - loss: 0.5379 - accuracy: 0.7015 - val_loss: 0.5415 - val_accuracy: 0.7020
Epoch 6/10
2089/2089 [==============================] - 4s 2ms/step - loss: 0.5378 - accuracy: 0.7007 - val_loss: 0.5450 - val_accuracy: 0.7004
Epoch 7/10
2089/2089 [==============================] - 5s 2ms/step - loss: 0.5372 - accuracy: 0.7017 - val_loss: 0.5414 - val_accuracy:

In [109]:
# 6. Đánh giá mô hình
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Loss: {loss}, Accuracy: {accuracy}')

2611/2611 [==============================] - 3s 1ms/step - loss: 0.5359 - accuracy: 0.7042
Loss: 0.5359011292457581, Accuracy: 0.7042108774185181
